# Exemple de profiling

Profiling et fonction *pdf*. Le profiling est utilisé pour mesurer le temps que passe un programme dans chaque fonction.

## Bizarrerie

C'est un exemple qui m'a été envoyé par un étudiant pendant l'été pour montrer que la fonction [pdf](http://docs.scipy.org/doc/scipy-0.16.0/reference/generated/scipy.stats.norm.html) est plus lente qu'une réimplémentation simple qui fait à la même chose.

In [1]:
import time
from scipy.stats import norm
import numpy as np

In [2]:
debut = time.time()
for i in range(10**3):
    norm(2, 3).pdf(4)
fin = time.time()
fin - debut

0.9644453525543213

In [3]:
def density(x, mean, sigma2):
    return np.exp(-((x - mean) ** 2) / (2 * sigma2)) / (2 * np.pi * sigma2) ** 0.5


debut = time.time()
for i in range(10**3):
    density(4, 2, 3)
fin = time.time()
fin - debut

0.001481771469116211

**Que se passe-t-il ?**

Tout d'abord la fonction [pdf](http://docs.scipy.org/doc/scipy-0.16.0/reference/generated/scipy.stats.norm.html) comme toute les fonctions des librairies numériques sont optimisées pour le calcul sur des matrices ou des vecteurs et non sur des nombres. Pour la suite, on utilise un profileur.

## Profiler

In [4]:
import cProfile, io, pstats, os, sys


def doprofile(func, filename, *l):
    pr = cProfile.Profile()
    pr.enable()  # début du profiling
    func(*l)  # appel de la fonction
    pr.disable()  # fin du profiling
    s = io.StringIO()
    ps = pstats.Stats(pr, stream=s).sort_stats("cumulative")
    ps.print_stats()
    rem = os.path.normpath(os.path.join(os.getcwd(), "..", "..", ".."))
    res = s.getvalue().replace(rem, "")
    res = res.replace(sys.base_prefix, "").replace("\\", "/")
    ps.dump_stats(filename)
    return res

In [5]:
import numpy

x = numpy.ones((10000000, 1)) * 4
x.shape

(10000000, 1)

In [6]:
debut = time.time()
y = norm.pdf(x)
fin = time.time()
print(fin - debut, y.shape, y[0])

0.6027283668518066 (10000000, 1) [0.00013383]


In [17]:
import os
import scipy

path = os.path.normpath(os.path.join(scipy.__file__, "..", "..", ".."))

r = doprofile(norm.pdf, "pdf.prof", x)
print(r.replace(path, ""))

         113 function calls in 0.450 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.178    0.178    0.450    0.450 /site-packages/scipy/stats/_distn_infrastructure.py:1958(pdf)
        1    0.000    0.000    0.142    0.142 /site-packages/scipy/stats/_continuous_distns.py:361(_pdf)
        1    0.142    0.142    0.142    0.142 /site-packages/scipy/stats/_continuous_distns.py:300(_norm_pdf)
        7    0.022    0.003    0.093    0.013 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    0.048    0.048 <__array_function__ internals>:177(place)
        1    0.000    0.000    0.048    0.048 /site-packages/numpy/lib/function_base.py:1912(place)
        1    0.048    0.048    0.048    0.048 {built-in method numpy.core._multiarray_umath._insert}
        1    0.028    0.028    0.028    0.028 /site-packages/scipy/stats/_distn_infrastructure.py:975(_support_mask)
   

In [8]:
def density(x, mean, sigma2):
    return np.exp(-((x - mean) ** 2) / (2 * sigma2)) / (2 * np.pi * sigma2) ** 0.5


debut = time.time()
y = density(x, 0.0, 1.0)
fin = time.time()
print(fin - debut, y.shape, y[0])

0.1882781982421875 (10000000, 1) [0.00013383]


In [9]:
r = doprofile(density, "pdf.prof", x, 0, 1)
print(r)

         2 function calls in 0.177 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.177    0.177    0.177    0.177 /tmp/ipykernel_29119/200996087.py:1(density)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}





Quand on regarde le code de la fonction, on s'aperçoit que la fonction perd du temps dans [argsreduce](https://github.com/scipy/scipy/blob/v0.17.1/scipy/stats/_distn_infrastructure.py#L521). Elle fait aussi d'autres choses comme regarder les valeurs manquantes. En guise de conclusion, lorsqu'une fonction gère trop de cas particuliers (type, valeurs), elle est nécessairement plus lente qu'une fonction qu'on implémente soi-même.